In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import sys
sys.path.append("../")

In [5]:
from lm_encoders_hf import MaskededLMEncoder
from torch.nn.functional import cosine_similarity

/data/yuwei/miniconda3/envs/llmembapi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model = MaskededLMEncoder(
    model_name_or_path="BrandonZYW/roberta-large-InBedder",
    mask_length=3
)

Embeddings will take up too much memory!


In [7]:
# set to last layer
model.set_output_value("avg_gen_layer_24")

In [8]:
instruction="Do they like animals?"
corpus = ["I like dog", "I like cat", "I hate dog"]
pattern = "### Input:\n{input}\n\n### Instruction:\n{instruction}\n\n### Response:"
corpus = [pattern.replace('{input}', s).replace('{instruction}', instruction) for s in corpus]
print(corpus)

['### Input:\nI like dog\n\n### Instruction:\nDo they like animals?\n\n### Response:', '### Input:\nI like cat\n\n### Instruction:\nDo they like animals?\n\n### Response:', '### Input:\nI hate dog\n\n### Instruction:\nDo they like animals?\n\n### Response:']


In [9]:
embeddings, generations = model.encode(
    corpus,
    batch_size=3,
    cache_dir=None, # useful when you want to reuse the embeddings
    return_generations=True, # useful if you want to look at your generations,
    convert_to_tensor=True
)

In [10]:
generations

["###Input:I like dog.### InstructionDo they like animals###ResponseYes't dog",
 "###Input:I like cat.### InstructionDo they like animals###ResponseYes't animals",
 "###Input:I hate dog.### InstructionDo they like animals###Responsen't dog"]

In [11]:
sim1 = cosine_similarity(embeddings[0], embeddings[1], dim=0)
sim2 = cosine_similarity(embeddings[0], embeddings[2], dim=0)
sim1 > sim2

tensor(True)

In [12]:
instruction="What animal are they talking about?"
corpus = ["I like dog", "I like cat", "I hate dog"]
pattern = "### Input:\n{input}\n\n### Instruction:\n{instruction}\n\n### Response:"
corpus = [pattern.replace('{input}', s).replace('{instruction}', instruction) for s in corpus]
print(corpus)

['### Input:\nI like dog\n\n### Instruction:\nWhat animal are they talking about?\n\n### Response:', '### Input:\nI like cat\n\n### Instruction:\nWhat animal are they talking about?\n\n### Response:', '### Input:\nI hate dog\n\n### Instruction:\nWhat animal are they talking about?\n\n### Response:']


In [13]:
embeddings, generations = model.encode(
    corpus,
    batch_size=3,
    cache_dir=None, # useful when you want to reuse the embeddings
    return_generations=True, # useful if you want to look at your generations,
    convert_to_tensor=True
)

In [14]:
generations

['###Input:"I like dog.### InstructionWhat animal are they talking about-###Response-dogoy dog',
 '###Input:"I like cat.### InstructionWhat animal are they talking about-###Response-citt cat',
 '###Input:"I hate dog.### InstructionWhat animal are they talking about-###Response-poy dog']

In [15]:
sim1 = cosine_similarity(embeddings[0], embeddings[1], dim=0)
sim2 = cosine_similarity(embeddings[0], embeddings[2], dim=0)
sim1 < sim2

tensor(True)